In [77]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import urllib

In [7]:
df = pd.read_csv('/Users/calebsmith/Library/Mobile Documents/com~apple~CloudDocs/Documents/Personal Data Projects/Sports Betting/Regular Season Games.csv')
df.drop(columns = 'Unnamed: 0', inplace = True)
df

,Home Team,Away Team,Home Moneyline,Away Moneyline,Home Spread,Home Spread Odds,Away Spread,Away Spread Odds,Over/Under,Over Odds,Under Odds
0,Chiefs,Ravens,-155.0,130.0,-3.0,108.0,3.0,112.0,47.0,110.0,110.0
1,Eagles,Packers,-125.0,105.0,-1.5,112.0,1.5,108.0,48.5,110.0,110.0
2,Bears,Titans,-218.0,180.0,-4.5,110.0,4.5,110.0,43.5,110.0,110.0
3,Bengals,Patriots,-410.0,320.0,-9.0,110.0,9.0,110.0,43.0,110.0,110.0
4,Bills,Cardinals,-360.0,285.0,-7.0,110.0,7.0,110.0,48.0,110.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...
266,Buccaneers,Saints,-135.0,114.0,-2.5,110.0,2.5,110.0,43.0,108.0,112.0
267,Cardinals,49ers,164.0,-198.0,4.0,108.0,-4.0,112.0,43.0,110.0,110.0
268,Raiders,Chargers,-105.0,-115.0,1.0,112.0,-1.0,108.0,43.5,110.0,110.0
269,Broncos,Chiefs,160.0,-192.0,4.0,110.0,-4.0,110.0,42.5,110.0,110.0


# Home Field Advantage (Points)

In [74]:
df['Home Spread'].describe()

count    271.000000
mean      -1.536900
std        3.792356
min      -11.500000
25%       -4.000000
50%       -2.000000
75%        1.500000
max        7.000000
Name: Home Spread, dtype: float64

In [78]:
def pfr_scrape(year):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    
    url = f'https://www.pro-football-reference.com/years/{year}/games.htm'
    headers={'User-Agent':user_agent,}
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
    
    return pd.read_html(data)

In [80]:
pfr = pfr_scrape(2023)
pfr_df = pfr[0]
pfr_df

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,Pts,Pts.1,YdsW,TOW,YdsL,TOL
0,1,Thu,2023-09-07,8:20PM,Detroit Lions,@,Kansas City Chiefs,boxscore,21,20,368,1,316,1
1,1,Sun,2023-09-10,1:00PM,Atlanta Falcons,NaN,Carolina Panthers,boxscore,24,10,221,0,281,3
2,1,Sun,2023-09-10,1:00PM,Cleveland Browns,NaN,Cincinnati Bengals,boxscore,24,3,350,2,142,0
3,1,Sun,2023-09-10,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,boxscore,31,21,342,2,280,3
4,1,Sun,2023-09-10,1:00PM,Washington Commanders,NaN,Arizona Cardinals,boxscore,20,16,248,3,210,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Division,Sun,2024-01-21,3:00PM,Detroit Lions,NaN,Tampa Bay Buccaneers,boxscore,31,23,391,0,408,2
299,Division,Sun,2024-01-21,6:30PM,Kansas City Chiefs,@,Buffalo Bills,boxscore,27,24,361,1,368,0
300,ConfChamp,Sun,2024-01-28,3:00PM,Kansas City Chiefs,@,Baltimore Ravens,boxscore,17,10,319,0,336,3
301,ConfChamp,Sun,2024-01-28,6:30PM,San Francisco 49ers,NaN,Detroit Lions,boxscore,34,31,413,1,442,1


In [82]:
pfr_df['Unnamed: 5'].unique()

array(['@', nan, 'N'], dtype=object)

In [94]:
home_away = pfr_df[pfr_df['Unnamed: 5'] != 'N'][['Unnamed: 5', 'Pts', 'Pts.1']]
home_away.rename(columns = {'Unnamed: 5': 'Winner', 'Pts': 'Winner Points', 'Pts.1': 'Loser Points'}, inplace = True)
home_away['Winner'].fillna('Home', inplace = True)
home_away.dropna(inplace = True)
home_away

/var/folders/0g/g5cy_rbj7kjgzj9ckj01rr1r0000gn/T/ipykernel_36551/2850980337.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  home_away['Winner'].fillna('Home', inplace = True)


,Winner,Winner Points,Loser Points
0,@,21,20
1,Home,24,10
2,Home,24,3
3,@,31,21
4,Home,20,16
...,...,...,...
297,Home,24,21
298,Home,31,23
299,@,27,24
300,@,17,10


In [95]:
home_away['Winner Points'].unique()

array(['21', '24', '31', '20', '25', '16', '30', '38', '17', '36', '40',
       '22', 'Pts', '34', '27', '37', '35', '26', '70', '18', '28', '15',
       '41', '23', '19', '48', '29', '42', '14', '39', '43', '13', '10',
       '33', '49', '32', '45', '12', '6', '3', '63', '9', '56'],
      dtype=object)

In [96]:
# ChatGPT wrote this function
def contains_pts(row):
    return row.astype(str).str.contains('Pts').any()

In [97]:
home_away = home_away[~home_away.apply(contains_pts, axis=1)]
home_away

,Winner,Winner Points,Loser Points
0,@,21,20
1,Home,24,10
2,Home,24,3
3,@,31,21
4,Home,20,16
...,...,...,...
297,Home,24,21
298,Home,31,23
299,@,27,24
300,@,17,10


In [98]:
# number of NFL Games
regular = 17*16
playoffs = 6 + 4 + 2
regular + playoffs

284

In [104]:
home_away['Winner Points'] = home_away['Winner Points'].astype(int)
home_away['Loser Points'] = home_away['Loser Points'].astype(int)
ha_group = home_away.groupby('Winner').sum()
ha_group

/var/folders/0g/g5cy_rbj7kjgzj9ckj01rr1r0000gn/T/ipykernel_36551/3947633823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_away['Winner Points'] = home_away['Winner Points'].astype(int)
/var/folders/0g/g5cy_rbj7kjgzj9ckj01rr1r0000gn/T/ipykernel_36551/3947633823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_away['Loser Points'] = home_away['Loser Points'].astype(int)


,Winner Points,Loser Points
Winner,,
@,3334,2141
Home,4492,2469


In [105]:
ha_matrix = ha_group.to_numpy()
ha_matrix

array([[3334, 2141],
       [4492, 2469]])

In [106]:
away_points = ha_matrix[0][0] + ha_matrix[1][1]
home_points = ha_matrix[0][1] + ha_matrix[1][0]

In [107]:
(home_points - away_points) / 284

2.9225352112676055

# Implied and Normalized Odds

In [25]:
def implied_odds(odds_series):
    odds_list = odds_series.to_list()
    implied_list = []

    for l in range(len(odds_list)):
        if odds_list[l] > 0:
            implied_list.append(100 / (odds_list[l] + 100))
        else:
            implied_list.append(odds_list[l] / (odds_list[l] - 100))

    implied_series = pd.Series(implied_list)
    
    return implied_series

In [29]:
implied_odds(df['Home Moneyline'])

0      0.607843
1      0.555556
2      0.685535
3      0.803922
4      0.782609
         ...   
266    0.574468
267    0.378788
268    0.512195
269    0.384615
270    0.692308
Length: 271, dtype: float64

In [43]:
df_implied = df.copy()
df_implied['Home Win %'] = round(100*implied_odds(df_implied['Home Moneyline']), 2)
df_implied['Away Win %'] = round(100*implied_odds(df_implied['Away Moneyline']), 2)
df_implied['Total Win %'] = df_implied['Home Win %'] + df_implied['Away Win %']
df_implied['Home Spread %'] = round(100*implied_odds(-df_implied['Home Spread Odds']), 2)
df_implied['Away Spread %'] = round(100*implied_odds(-df_implied['Away Spread Odds']), 2)
df_implied['Total Spread %'] = df_implied['Home Spread %'] + df_implied['Away Spread %']
df_implied['Over %'] = round(100*implied_odds(-df_implied['Over Odds']), 2)
df_implied['Under %'] = round(100*implied_odds(-df_implied['Under Odds']), 2)
df_implied['Over/Under %'] = df_implied['Over %'] + df_implied['Under %']
df_implied = df_implied[['Home Team', 'Away Team', 'Home Win %', 'Away Win %', 'Total Win %', 'Home Spread', 'Home Spread %', 'Away Spread', 'Away Spread %', 'Total Spread %', 'Over/Under', 'Over %', 'Under %', 'Over/Under %']]
df_implied

,Home Team,Away Team,Home Win %,Away Win %,Total Win %,Home Spread,Home Spread %,Away Spread,Away Spread %,Total Spread %,Over/Under,Over %,Under %,Over/Under %
0,Chiefs,Ravens,60.78,43.48,104.26,-3.0,51.92,3.0,52.83,104.75,47.0,52.38,52.38,104.76
1,Eagles,Packers,55.56,48.78,104.34,-1.5,52.83,1.5,51.92,104.75,48.5,52.38,52.38,104.76
2,Bears,Titans,68.55,35.71,104.26,-4.5,52.38,4.5,52.38,104.76,43.5,52.38,52.38,104.76
3,Bengals,Patriots,80.39,23.81,104.20,-9.0,52.38,9.0,52.38,104.76,43.0,52.38,52.38,104.76
4,Bills,Cardinals,78.26,25.97,104.23,-7.0,52.38,7.0,52.38,104.76,48.0,52.38,52.38,104.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Buccaneers,Saints,57.45,46.73,104.18,-2.5,52.38,2.5,52.38,104.76,43.0,51.92,52.83,104.75
267,Cardinals,49ers,37.88,66.44,104.32,4.0,51.92,-4.0,52.83,104.75,43.0,52.38,52.38,104.76
268,Raiders,Chargers,51.22,53.49,104.71,1.0,52.83,-1.0,51.92,104.75,43.5,52.38,52.38,104.76
269,Broncos,Chiefs,38.46,65.75,104.21,4.0,52.38,-4.0,52.38,104.76,42.5,52.38,52.38,104.76


In [45]:
df_implied['Total Spread %'].describe()

count    271.000000
mean     104.738413
std        0.139082
min      104.550000
25%      104.750000
50%      104.760000
75%      104.760000
max      106.780000
Name: Total Spread %, dtype: float64

In [46]:
df_implied['Over/Under %'].describe()

count    271.000000
mean     104.757712
std        0.004208
min      104.750000
25%      104.760000
50%      104.760000
75%      104.760000
max      104.760000
Name: Over/Under %, dtype: float64

In [47]:
df_implied['Total Win %'].describe()

count    271.000000
mean     104.341292
std        0.180126
min      104.080000
25%      104.220000
50%      104.280000
75%      104.350000
max      104.760000
Name: Total Win %, dtype: float64

In [53]:
df_normalized = pd.DataFrame({'Home Team': df['Home Team'], 'Away Team': df['Away Team']})
df_normalized['Home Win %'] = df_implied['Home Win %'] / (df_implied['Total Win %']/100)
df_normalized['Away Win %'] = df_implied['Away Win %'] / (df_implied['Total Win %']/100)
df_normalized

,Home Team,Away Team,Home Win %,Away Win %
0,Chiefs,Ravens,58.296566,41.703434
1,Eagles,Packers,53.248994,46.751006
2,Bears,Titans,65.749089,34.250911
3,Bengals,Patriots,77.149712,22.850288
4,Bills,Cardinals,75.083949,24.916051
...,...,...,...,...
266,Buccaneers,Saints,55.144941,44.855059
267,Cardinals,49ers,36.311350,63.688650
268,Raiders,Chargers,48.916054,51.083946
269,Broncos,Chiefs,36.906247,63.093753


In [54]:
df_normalized.sort_values('Home Win %')

,Home Team,Away Team,Home Win %,Away Win %
170,Panthers,Chiefs,25.595010,74.404990
210,Giants,Ravens,29.498610,70.501390
209,Panthers,Cowboys,29.498610,70.501390
56,Panthers,Bengals,29.984648,70.015352
22,Vikings,49ers,30.921116,69.078884
...,...,...,...,...
3,Bengals,Patriots,77.149712,22.850288
160,49ers,Seahawks,77.149712,22.850288
72,49ers,Cardinals,77.679685,22.320315
129,Ravens,Broncos,80.001920,19.998080


### Normalized Implied Home Team Win %

In [73]:
df_normalized['Home Win %'].describe()

count    271.000000
mean      54.725650
std       11.781059
min       25.595010
25%       46.751006
50%       54.273094
75%       63.093753
max       82.231603
Name: Home Win %, dtype: float64

Normalized isn't actually as useful as implied. The implied odds are what you need to beat to beat the casino.

In [55]:
moneyline = df_implied[['Home Team', 'Away Team', 'Home Win %', 'Away Win %']]
moneyline

,Home Team,Away Team,Home Win %,Away Win %
0,Chiefs,Ravens,60.78,43.48
1,Eagles,Packers,55.56,48.78
2,Bears,Titans,68.55,35.71
3,Bengals,Patriots,80.39,23.81
4,Bills,Cardinals,78.26,25.97
...,...,...,...,...
266,Buccaneers,Saints,57.45,46.73
267,Cardinals,49ers,37.88,66.44
268,Raiders,Chargers,51.22,53.49
269,Broncos,Chiefs,38.46,65.75


In [56]:
moneyline.sort_values('Home Win %')

,Home Team,Away Team,Home Win %,Away Win %
170,Panthers,Chiefs,26.67,77.53
210,Giants,Ravens,30.77,73.54
209,Panthers,Cowboys,30.77,73.54
56,Panthers,Bengals,31.25,72.97
22,Vikings,49ers,32.26,72.07
...,...,...,...,...
3,Bengals,Patriots,80.39,23.81
160,49ers,Seahawks,80.39,23.81
72,49ers,Cardinals,80.95,23.26
129,Ravens,Broncos,83.33,20.83


In [66]:
team_win_pct = moneyline.drop(columns = 'Away Team').groupby('Home Team').mean()[['Home Win %']]
team_win_pct['Home Games'] = moneyline.drop(columns = 'Away Team').groupby('Home Team').count()['Home Win %']
#team_win_pct['Home Games'] = team_home_games
team_win_pct['Away Win %'] = moneyline.drop(columns = 'Home Team').groupby('Away Team').mean()['Away Win %']
team_win_pct['Away Games'] = moneyline.drop(columns = 'Home Team').groupby('Away Team').count()['Away Win %']
team_win_pct['Total Games'] = team_win_pct['Home Games'] + team_win_pct['Away Games']
team_win_pct['Total Win %'] = (team_win_pct['Home Win %']*team_win_pct['Home Games'] + team_win_pct['Away Win %']*team_win_pct['Away Games']) / team_win_pct['Total Games']
team_win_pct

,Home Win %,Home Games,Away Win %,Away Games,Total Games,Total Win %
Home Team,,,,,,
49ers,72.460000,9,62.961250,8,17,67.990000
Bears,60.343333,9,45.716250,8,17,53.460000
Bengals,69.948750,8,55.923333,9,17,62.523529
Bills,65.647500,8,52.525556,9,17,58.700588
Broncos,45.453750,8,32.930000,9,17,38.823529
Browns,56.408750,8,51.727778,9,17,53.930588
Buccaneers,53.214444,9,42.583750,8,17,48.211765
Cardinals,49.485556,9,37.047500,8,17,43.632353
Chargers,55.750000,8,50.432222,9,17,52.934706


In [68]:
games17 = team_win_pct[team_win_pct['Total Games'] == 17]
games17['Implied Wins'] = games17['Total Games'] * games17['Total Win %'] / 100
games17

/var/folders/0g/g5cy_rbj7kjgzj9ckj01rr1r0000gn/T/ipykernel_36551/3410766752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games17['Implied Wins'] = games17['Total Games'] * games17['Total Win %'] / 100


,Home Win %,Home Games,Away Win %,Away Games,Total Games,Total Win %,Implied Wins
Home Team,,,,,,,
49ers,72.460000,9,62.961250,8,17,67.990000,11.5583
Bears,60.343333,9,45.716250,8,17,53.460000,9.0882
Bengals,69.948750,8,55.923333,9,17,62.523529,10.6290
Bills,65.647500,8,52.525556,9,17,58.700588,9.9791
Broncos,45.453750,8,32.930000,9,17,38.823529,6.6000
Browns,56.408750,8,51.727778,9,17,53.930588,9.1682
Buccaneers,53.214444,9,42.583750,8,17,48.211765,8.1960
Cardinals,49.485556,9,37.047500,8,17,43.632353,7.4175
Chargers,55.750000,8,50.432222,9,17,52.934706,8.9989


In [71]:
games17.sort_values('Implied Wins', ascending = False)

,Home Win %,Home Games,Away Win %,Away Games,Total Games,Total Win %,Implied Wins
Home Team,,,,,,,
49ers,72.460000,9,62.961250,8,17,67.990000,11.5583
Chiefs,72.715000,8,63.076667,9,17,67.612353,11.4941
Ravens,70.683750,8,57.642222,9,17,63.779412,10.8425
Bengals,69.948750,8,55.923333,9,17,62.523529,10.6290
Lions,67.672222,9,53.311250,8,17,60.914118,10.3554
Eagles,65.524444,9,54.512500,8,17,60.342353,10.2582
Jets,65.272500,8,54.665556,9,17,59.657059,10.1417
Cowboys,62.746667,9,55.915000,8,17,59.531765,10.1204
Bills,65.647500,8,52.525556,9,17,58.700588,9.9791
